### Library Imports

In [ ]:
%matplotlib inline
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
from sklearn.datasets import make_classification
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer
from IPython.display import clear_output
seed = 42

### Class initialisation

To recap from the lecture, creating a feedforward neural network and training it involves the following steps:

1. **Initialise Layers:**
   - For each layer, initialise a weight matrix and a bias vector:
      - The weight matrix (**W**) size is (input features size, output features size).
      - The bias vector (**b**) size is (1, output features size).

2. **Define Activation Functions for Hidden Layers:**
   - Choose activation functions for hidden layers. Common choices include:
      - [Sigmoid](https://en.wikipedia.org/wiki/Sigmoid_function)
      - [Softmax](https://en.wikipedia.org/wiki/Softmax_function)
      - [ReLU](https://en.wikipedia.org/wiki/Rectifier_(neural_networks)

3. **Define Forward Propagation Function:**
   - Implement a function that computes the output of the neural network given an input by applying activation functions to the weighted sums of inputs at each layer.

4. **Define Loss Function:**
   - Choose a loss function that measures the distance between predicted and gold outputs. Common choices include mean squared error for regression tasks and cross-entropy loss for classification tasks.

5. **Define Backward Propagation Function:**
   - Implement a function that computes the gradients of the loss with respect to the weights and biases by applying the chain rule of calculus.

6. **Define Training Loop:**
   - Iterate through the training data multiple times, performing the following in each iteration:
      - Forward pass to compute predictions.
      - Compute loss using the chosen loss function.
      - Backward pass to compute gradients.
      - Update weights and biases using an optimisation algorithm (e.g., gradient descent).
      - Repeat until the loss converges or a predefined number of iterations are reached.

It's good practice to implement neural network architectures using the class data structure in Python for reusability. This allows training multiple instances with different hyperparameters and facilitates debugging in case of issues. Following this practice, below we implement our feedforward neural network as a class, incorporating the steps above.

In [ ]:
class FeedforwardNeuralNetwork:
    # Step 1: Initialise weights and biases
    def __init__(self, input_size: int, hidden_size1: int, hidden_size2: int, output_size: int) -> None:
        '''
        Initialises the neural network with random weights and zero biases.

        Args:
          input_size (int): Number of features in the input data.
          hidden_size1 (int): Number of neurons in the first hidden layer.
          hidden_size2 (int): Number of neurons in the second hidden layer.
          output_size (int): Number of classes in the output layer.
        '''

        # Layer 1
        # Layer 1 weights matrix: (input_size, hidden_size1)
        self.W1 = np.random.randn(input_size, hidden_size1)
        print("Weights layer 1 shape:", self.W1.shape) # -> (1500, 64)
        self.b1 = np.zeros((1, hidden_size1))

        # Layer 2
        # Layer 2 weights matrix: (hidden_size1, hidden_size2)
        self.W2 = np.random.randn(hidden_size1, hidden_size2)
        print("Weights layer 2 shape:", self.W2.shape) # -> (64, 32)
        self.b2 = np.zeros((1, hidden_size2))

        # Layer 3 (Output layer)
        # Layer 3 weights matrix: (hidden_size2, output_size)
        self.W3 = np.random.randn(hidden_size2, output_size)
        print("Weights layer 3 shape:", self.W3.shape) # -> (32, 3)
        self.b3 = np.zeros((1, output_size))


    # Step 2: Activation Functions
    # 2.1: activation functions
    def sigmoid(self, z: np.ndarray) -> np.ndarray:
        """
          Sigmoid function that gives us the predictions of the given samples.

          Args:
              Z (array): each row contains z = b + w_1 x_1 + w_2 x_2 + ... + w_n x_n

          Returns:
              np.ndarray: array containing a prediction for each sample.
        """
        return 1 / (1 + np.exp(-z))

    def softmax(self, z: np.ndarray) -> np.ndarray:
        """
          Softmax function that gives us the predictions of the given samples.

          Args:
              Z (array): each row contains z = b + w_1 x_1 + w_2 x_2 + ... + w_n x_n

          Returns:
              np.ndarray: array containing a prediction for each sample.
        """
        exp_z = np.exp(z - np.max(z, axis=1, keepdims=True))
        return exp_z / np.sum(exp_z, axis=1, keepdims=True)

    # 2.2: derivatives of activation functions
    def d_sigmoid(self, x: np.ndarray) -> np.ndarray:
      return x * (1 - x)

    # Step 3: Perform Forward propagation
    def forward(self, X: np.ndarray, epoch: int=0) -> np.ndarray:
        '''
        Performs forward propagation for the neural network.

        Args:
            X (numpy.ndarray): Input data with dimensions (num_samples, num_features).

        Returns:
            numpy.ndarray: Output predictions after passing through the network's layers.
        '''

        ## INSERT YOUR CODE HERE ##

        # Layer 1

        # Compute Z for Layer 1 using input data and Layer 1 weights
        # X: (num_train_samples, num_features) x weights layer 1: (input_size, hidden_size1) -> Z1: (num_train_samples, hidden_size1)
        Z1 = np.dot(X, self.W1) + self.b1

        # Compute the output of Layer 1 using the activation function of Layer 1 and Layer 1 Z
        # Z1: (num_train_samples, hidden_size1) -> a1: (num_train_samples, hidden_size1)
        self.a1 = self.sigmoid(Z1)

        if epoch == 0:
          print("Z1 shape (num_train_samples, hidden_size1):", Z1.shape)  # -> (4500, 64)
          print("a1 shape (num_train_samples, hidden_size1):", self.a1.shape)  # -> (4500, 64)


        # Layer 2

        # Compute Z for Layer 2 using output of Layer 1 and Layer 2 weights
        # a1: (num_train_samples, hidden_size1) x weights layer 2: (hidden_size1, hidden_size2) -> Z2: (num_train_samples, hidden_size2)
        Z2 = np.dot(self.a1, self.W2) + self.b2

        # Compute the output of Layer 2 using the activation function of Layer 2 and Layer 2 Z
        # Z2: (num_train_samples, hidden_size2) -> a2: (num_train_samples, hidden_size2)
        self.a2 = self.sigmoid(Z2)

        if epoch == 0:
          print("Z2 shape (num_train_samples, hidden_size2):", Z2.shape)  # -> (4500, 32)
          print("a2 shape (num_train_samples, hidden_size2):", self.a2.shape)  # -> (4500, 32)


        # Layer 3 (Output Layer)

        # Compute Z for Layer 3 using output of Layer 2 and Layer 3 weights
        # a2: (num_train_samples, hidden_size2) x weights layer 3: (hidden_size2, output_size) -> Z3: (num_train_samples, output_size)
        Z3 = np.dot(self.a2, self.W3) + self.b3

        # Compute the output of Layer 3 (output of the network) using the activation function of Layer 3 and Layer 3 Z
        # Z3: (num_train_samples, output_size) -> a3: (num_train_samples, output_size)
        self.a3 = self.softmax(Z3)

        if epoch == 0:
          print("Z3 shape (num_train_samples, output_size):", Z3.shape)  # -> (4500, 3)
          print("a3 shape (num_train_samples, output_size):", self.a3.shape)  # -> (4500, 3)

        ## END OF YOUR CODE ##

        return self.a3


    # Step 4: Cross-Entropy Loss Calculation
    def cross_entropy_loss(self, y_true: np.ndarray, y_pred: np.ndarray) -> float:
        '''
        Computes the cross-entropy loss between the true labels and predicted probabilities.

        Args:
          y_true (numpy.ndarray): True class labels with dimensions (num_samples,).
          y_pred (numpy.ndarray): Predicted probabilities with dimensions (num_samples, num_classes).

        Returns:
          float: Cross-entropy loss.
        '''
        num_samples = len(y_true)
        loss = -np.sum(np.log(y_pred[np.arange(num_samples), y_true])) / num_samples
        return loss


    # Step 6: Perform Backward propagation
    def backward(self, X: np.ndarray, y_true: np.ndarray, learning_rate: float=0.01, epoch: int=1) -> None:
        '''
        Computes the cross-entropy loss between the true labels and predicted probabilities.

        Args:
          X (numpy.ndarray): Samples in the dataset.
          y_true (numpy.ndarray): True class labels with dimensions (num_samples,).
          learning_rate (float): Learning rate used to update weights. Deafulto to 0.01.
          epoch (int): current epoch during training, used to print matrix weights only for first epoch. Default to 1.
        '''

        num_samples = len(X)

        # 5.1 Compute Gradients for Layer 3 (Output Layer) (d_a3, d_W3, d_b3):
        '''
        d_a3: Calculate the error at the output layer by subtracting 1
         from the predicted probabilities corresponding to the true class.

        d_W3: Compute the gradient of the loss with respect to the weights connecting
         the third layer (output layer) to the second layer.

        d_b3: Compute the gradient of the loss with respect to the biases of the output layer.
        '''
        d_a3 = self.a3.copy()
        d_a3[np.arange(num_samples), y_true] -= 1
        d_L3 = d_a3/ num_samples

        d_W3 = np.dot(self.a2.T, d_L3)
        d_b3 = np.sum(d_L3, axis=0, keepdims=True)
        if epoch == 0:
          print("d_a3 shape (num_train_samples, output_size):", d_a3.shape)  # -> (4500, 3)
          print("d_L3 shape (num_train_samples, output_size):", d_L3.shape)  # -> (4500, 3)
          print("d_W3 shape (hidden_size2, output_size):", d_W3.shape)  # -> (32, 3)
          print("d_b3 shape (1, output_size):", d_b3.shape)  # -> (1, 3)

        # 5.2 Compute Gradients for Layer 2 (d_a2, d_W2, d_b2):
        '''
        d_a2: Calculate the error at the hidden layer .

        d_W2: Compute the gradient of the layer 3 output with respect to the weights connecting
         the second layer (hidden layer) to the first layer (input layer).

        d_b2: Compute the gradient of the layer 3 output with respect to the biases of the hidden layer.
        '''
        d_a2 = np.dot(d_L3, self.W3.T)
        d_L2 = d_a2 * (self.d_sigmoid(self.a2))

        d_W2 = np.dot(self.a1.T, d_L2)
        d_b2 = np.sum(d_L2, axis=0, keepdims=True)
        if epoch == 0:
          print("d_a2 shape (num_train_samples, hidden_size2):", d_a2.shape)  # -> (4500, 32)
          print("d_L2 shape (num_train_samples, hidden_size2):", d_L2.shape)  # -> (4500, 32)
          print("d_W2 shape (hidden_size1, hidden_size2):", d_W2.shape)  # -> (64, 32)
          print("d_b2 shape (1, hidden_size2):", d_b2.shape)  # -> (1, 32)

        # 5.3 Compute Gradients for Layer 1 (d_a1, d_W1, d_b1):
        d_a1 = np.dot(d_L2, self.W2.T)
        d_L1 = d_a1 * (self.d_sigmoid(self.a1))

        d_W1 = np.dot(X.T, d_L1)
        d_b1 = np.sum(d_L1, axis=0, keepdims=True)
        if epoch == 0:
          print("d_a1 shape (num_train_samples, hidden_size1):", d_a1.shape)  # -> (4500, 64)
          print("d_L1 shape (num_train_samples, hidden_size1):", d_L1.shape)  # -> (4500, 64)
          print("d_W1 shape (input_size, hidden_size2):", d_W1.shape)  # -> (1500, 64)
          print("d_b1 shape (1, input_size):", d_b1.shape)  # -> (1, 64)

        # 5.4 Update weights and biases
        self.W1 -= learning_rate * d_W1
        self.b1 -= learning_rate * d_b1
        self.W2 -= learning_rate * d_W2
        self.b2 -= learning_rate * d_b2
        self.W3 -= learning_rate * d_W3
        self.b3 -= learning_rate * d_b3

    # Step 6: Training Loop
    def train(self, X: np.ndarray, y_true: np.ndarray, epochs: int=100, learning_rate: float=0.01) -> None:
        '''
        Trains the neural network using the specified training data.

        Args:
            X (numpy.ndarray): Input data with dimensions (num_samples, num_features).
            y_true (numpy.ndarray): True class labels with dimensions (num_samples,).
            epochs (int): Number of training epochs. Default to 100.
            learning_rate (float): Learning rate for weight updates. Default to 0.01.
        '''
        self.losses = []

        # tqdm is a function that given a list, the total number of iterations and a description,
        # returns a progressbar showing the time execution
        # pbar will be updated to include the loss at each epoch
        pbar = tqdm(range(epochs), total=epochs, desc="Training")
        for epoch in pbar:
              # 6.1 Forward pass
              y_pred = self.forward(X, epoch)

              # 6.2 Compute loss
              loss = self.cross_entropy_loss(y_true, y_pred)
              self.losses.append(loss)
              # updated the progressbar to include the loss of this epoch
              pbar.set_description(f'Training - Epoch {epoch + 1}/{epochs}, Loss: {loss:0.03f}') #0.03f to display the first decimal points

              # 6.3 Backward pass
              self.backward(X, y_true, learning_rate, epoch)


    def show_learning_curve(self) -> None:
        '''
        Displays the learning curve (loss curve) during training.
        '''
        plt.plot(self.losses)
        plt.xlabel('Epochs') # x axis label
        plt.ylabel('Cross-Entropy Loss') # y axis label
        plt.show()


    def test(self, X: np.ndarray) -> np.ndarray:
        '''
        Calculates the predictions for the given samples.

        Args:
            X (numpy.ndarray): Input data with dimensions (num_samples, num_features).

        Returns:
            numpy.ndarray: Output predictions after passing through the network's layers.
        '''
        y_pred = self.forward(X)
        return y_pred